In [29]:
%matplotlib inline

from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pylab as plt

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
BD = pd.read_csv('/content/drive/MyDrive/Ass2_FML/UniversalBank.csv',delimiter=',')
print(BD.head())

   ID  Age  Experience  Income  ZIP Code  Family  CCAvg  Education  Mortgage  \
0   1   25           1      49     91107       4    1.6          1         0   
1   2   45          19      34     90089       3    1.5          1         0   
2   3   39          15      11     94720       1    1.0          1         0   
3   4   35           9     100     94112       1    2.7          2         0   
4   5   35           8      45     91330       4    1.0          2         0   

   Personal Loan  Securities Account  CD Account  Online  CreditCard  
0              0                   1           0       0           0  
1              0                   1           0       0           0  
2              0                   0           0       0           0  
3              0                   0           0       0           0  
4              0                   0           0       0           1  


In [32]:
df_train= BD[['Personal Loan','CreditCard','Online']]

**Partition the data into training (60%) and validation (40%) sets.**

In [33]:
# split the data into train and test set
train, test = train_test_split(df_train, test_size=0.4, random_state=42, shuffle=True)

**Normalize the train data**

In [34]:
from sklearn import preprocessing
outcome = 'Personal Loan'
predictors = list(df_train)
predictors.remove(outcome)
Scaler = preprocessing.StandardScaler()
Scaler.fit(df_train[predictors])

StandardScaler()

**(A)Create a pivot table for the training data with Online as a column variable, CC as a row variable, and Loan as a secondary row variable. The values inside the table should convey the count.**

In [35]:
Total_byCell = pd.pivot_table(train, index = ['CreditCard','Personal Loan'], values = 'Online', aggfunc = 'sum')
print(Total_byCell)


                          Online
CreditCard Personal Loan        
0          0                1141
           1                 129
1          0                 481
           1                  46


**(B)Consider the task of classifying a customer who owns a bank credit card and is actively using online banking services. Looking at the pivot table, what is the probability that this customer will accept the loan offer? [This is the probability of loan acceptance (Loan = 1) conditional on having a bank credit card (CC = 1) and being an active user of online banking services (Online = 1)].**

In [36]:
train.pivot_table(index = ['CreditCard','Personal Loan','Online'], aggfunc=len,fill_value=0)

CreditCard  Personal Loan  Online
0           0              0          780
                           1         1141
            1              0           70
                           1          129
1           0              0          319
                           1          481
            1              0           34
                           1           46
dtype: int64

**46/46+481=0.088 which is the probability**

**(C)Create two separate pivot tables for the training data.**

**One will have Loan (rows) as a function of Online (columns)**

In [37]:
pivot1=pd.pivot_table(train,index=['Personal Loan'],values='Online',aggfunc=len)
pivot1

,Online
Personal Loan,
0,2721
1,279


**Loan (rows) as a function of CC.**

In [38]:
pivot2=pd.pivot_table(train,index=['Personal Loan'],values='Online',aggfunc=len)
pivot2

,Online
Personal Loan,
0,2721
1,279


**(D)Compute the following quantities [P(A | B) means “the probability ofA given B”]:**

**i. P(CC = 1 | Loan = 1)**

In [39]:
train.pivot_table(index = ['CreditCard','Personal Loan'],aggfunc=len,fill_value=0)

Online
CreditCard Personal Loan        
0          0                1921
           1                 199
1          0                 800
           1                  80

**80/80+199=0.286**

**ii. P(Online = 1 | Loan = 1)**

In [40]:
train.pivot_table(index = ['Online','Personal Loan'], aggfunc=len,fill_value=0)

CreditCard
Online Personal Loan            
0      0                    1099
       1                     104
1      0                    1622
       1                     175

**175/175+104=0.627**

**iii. P(Loan = 1) (the proportion of loan acceptors)**

In [41]:
train.pivot_table(index = ['Personal Loan'], aggfunc=len,fill_value=0)


,CreditCard,Online
Personal Loan,,
0,2721,2721
1,279,279


**279/279+2721=0.093**

**iv. P(CC = 1 | Loan = 0)**

In [42]:
train.pivot_table(index = ['Personal Loan','CreditCard'], aggfunc=len,fill_value=0)


Online
Personal Loan CreditCard        
0             0             1921
              1              800
1             0              199
              1               80

**800/800+1921=0.294**

**v. P(Online = 1 | Loan = 0)**

In [43]:
train.pivot_table(index = ['Personal Loan','Online'], aggfunc=len,fill_value=0)


CreditCard
Personal Loan Online            
0             0             1099
              1             1622
1             0              104
              1              175

**1622/1622+1099=0.596**

**vi. P(Loan = 0)**

In [44]:
train.pivot_table(index = ['Personal Loan'], aggfunc=len,fill_value=0)


,CreditCard,Online
Personal Loan,,
0,2721,2721
1,279,279


**2721/2721+279=0.907**

**(E)Use the quantities computed above to compute the naive Bayes probability(Loan = 1 | CC = 1, Online = 1).**

**NAIVE BAYES CALCULATION**

0.2860.0930.627/[(0.2860.6270.093)+(0.2940.5960.907)=0.081

**(F) The value from the pivot table i.e 0.88 seems to be more accurate when compared to the value obtained from Naive Bayes probabilty,because (B) uses direct calculation from count whereas (E) uses probabilities of the each of those counts,thus (B) is more accurate.**




**(G) Which of the entries in this table are needed for computing P(Loan = 1 | CC = 1, Online = 1)?**

**The entries needed for computing P(Loan = 1 | CC=1,Online=1) that correspond on both tables are the entires of CC=1 and Online=1. This is because in order for the situation to be Loan=1, there also needs to be CC=1, Online=1.**

In [45]:
from sklearn.model_selection import train_test_split as tts
X=BD[['Online','CreditCard']]
y=BD["Personal Loan"]

**Creating Naive Bayes Classifier Model**

In [46]:
# split into training and validation
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.40, random_state=1)

# run naive Bayes
nb = MultinomialNB(alpha=0.01)
nb.fit(X_train, y_train)

# predict probabilities
predProb_train = nb.predict_proba(X_train)
predProb_valid = nb.predict_proba(X_valid)

# predict class membership
y_valid_pred = nb.predict(X_valid)
y_train_pred = nb.predict(X_train)


**Run naive Bayes on the data. Examine the model output on training data, and find the entry that corresponds to P(Loan = 1 | CC = 1, Online = 1).Compare this to the number you obtained in (E).**

In [47]:
# Subset a specific set
df = pd.concat([pd.DataFrame({'actual': y_valid, 'predicted': y_valid_pred}),
                pd.DataFrame(predProb_valid, index=y_valid.index)], axis=1)
mask = ((X_valid.Online == 1) & (X_valid.CreditCard == 1))

print(df[mask])

      actual  predicted         0         1
932        0          0  0.904419  0.095581
1132       0          0  0.904419  0.095581
3289       0          0  0.904419  0.095581
348        1          0  0.904419  0.095581
2971       0          0  0.904419  0.095581
...      ...        ...       ...       ...
4705       1          0  0.904419  0.095581
2429       0          0  0.904419  0.095581
2684       0          0  0.904419  0.095581
3401       0          0  0.904419  0.095581
1239       0          0  0.904419  0.095581

[356 rows x 4 columns]


**The value obtained here is 0.095 and the value obtained in (E) is 0.081,the result is almost the same,with only a minute difference because of the rounding,anyways the difference wont affect the rank order of the output**

**Accuracy**

In [48]:
###Evaluating Prediction Accuracy
from sklearn import metrics
print("Acc %:",metrics.accuracy_score(y_valid,y_valid_pred )*100)

Acc %: 90.35
